In [1]:
"""
Adapted from: https://github.com/aliasvishnu/EEGNet/blob/master/EEGNet-PyTorch.ipynb

Original paper - https://arxiv.org/abs/1611.08024

Please reach out to me if you spot an error
"""

'\nAdapted from: https://github.com/aliasvishnu/EEGNet/blob/master/EEGNet-PyTorch.ipynb\n\nOriginal paper - https://arxiv.org/abs/1611.08024\n\nPlease reach out to me if you spot an error\n'

In [2]:
import pandas as pd 
import numpy as np
from numpy import array
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from ipynb.fs.full.Data_Processing import extract_test_number
from sklearn import preprocessing
import torch.utils.data as data_utils

In [3]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 8), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, 1)
        
    def forward(self, x):
        # Layer 1
        x = x.float()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)

        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)

        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)

        # FC Layer
        x = x.view(-1, 4*2*7)
        x = F.sigmoid(self.fc1(x))
        return x

In [4]:
net = EEGNet().cuda(0)
print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 8)).cuda(0))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())

tensor([[0.4335]], device='cuda:0', grad_fn=<SigmoidBackward>)


/cs/home/ybk1/python/lib/python3.7/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


**Evaluate function returns values of different criteria like accuracy, precision etc.**
In case you face memory overflow issues, use batch size to control how many samples get evaluated at one time. Use a batch_size that is a factor of length of samples. This ensures that you won't miss any samples.

In [5]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results

**Generate random data**
    
*Data format:*

Datatype - float32 (both X and Y)

X.shape - (#samples, 1, #timepoints, #channels)

Y.shape - (#samples)

In [6]:
X_train = np.random.rand(100, 1, 120, 64).astype('float32') # np.random.rand generates between [0, 1)
y_train = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.

X_val = np.random.rand(100, 1, 120, 64).astype('float32')
y_val = np.round(np.random.rand(100).astype('float32'))

X_test = np.random.rand(100, 1, 120, 64).astype('float32')
y_test = np.round(np.random.rand(100).astype('float32'))

### Loading the Data

In [7]:
training_files = pd.read_csv("clean_trainingfiles.csv")

In [8]:
user_1 = array(training_files[training_files['user'] == 1]["path"])

In [9]:
attention = "EEG_attention_dataset.csv"
effort = "EEG_effort_dataset.csv"
interest = "EEG_interest_dataset.csv"

In [10]:
user_1_tests = {}

for test in user_1:
    datasets = [test + "/" + attention, test + "/" + interest, test + "/" + effort]
    user_1_tests[extract_test_number(test)] = datasets


    

In [11]:
test_10_attention = pd.read_csv(user_1_tests[10][0])
test_10_attention 

,C1,C2,C3,C4,C5,C6,C7,C8,Timestamp,attention
0,208520.203125,-266677.78125,249952.484375,229751.750000,204797.812500,184424.953125,190706.890625,223252.703125,1.459584e+06,5.0
1,208641.656250,-266710.84375,250072.890625,229724.140625,204892.375000,184429.046875,190821.140625,223333.437500,1.459619e+06,5.0
2,208745.796875,-266736.40625,250180.078125,229704.781250,204981.453125,184438.437500,190925.000000,223416.156250,1.459630e+06,5.0
3,208799.625000,-266741.53125,250235.531250,229702.687500,205035.140625,184457.234375,190988.031250,223475.156250,1.459649e+06,5.0
4,208788.093750,-266730.53125,250224.328125,229715.281250,205030.843750,184471.921875,190984.796875,223486.359375,1.459660e+06,5.0
...,...,...,...,...,...,...,...,...,...,...
2395,208734.625000,-266508.21875,250261.953125,230237.031250,205071.859375,184635.234375,191025.703125,223626.453125,1.515933e+06,4.0
2396,208765.671875,-266517.28125,250284.265625,230226.828125,205075.765625,184626.453125,191036.156250,223619.921875,1.515959e+06,4.0
2397,208866.812500,-266544.81250,250380.109375,230203.843750,205141.765625,184625.546875,191120.078125,223670.562500,1.516002e+06,4.0
2398,209003.328125,-266569.15625,250514.109375,230186.390625,205246.328125,184640.578125,191244.343750,223763.437500,1.516039e+06,4.0


In [12]:
t10_AT_X_train = test_10_attention.iloc[:,:-2]
t10_AT_y_train = array(test_10_attention.iloc[:,-1])

print("X_train shape {0}\n y_train shape {1}".format(t10_AT_X_train.shape, t10_AT_y_train.shape))

X_train shape (2400, 8)
 y_train shape (2400,)


### Scale the data

In [13]:
t10_AT_X_train=preprocessing.scale(t10_AT_X_train)

### Load the data

In [14]:
features = torch.Tensor(t10_AT_X_train)
targets = torch.Tensor(t10_AT_y_train)

In [15]:
import torch.utils.data as data_utils

train = data_utils.TensorDataset(features, targets)
train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)

In [70]:
# t10_AT_X_train = np.reshape(array(t10_AT_X_train), (2400, 1, 1, 8))

In [23]:
# t10_AT_y_train = array(t10_AT_y_train)

In [63]:
print("X shape {0} \n y shape {1}".format(t10_AT_X_train.shape, t10_AT_y_train.shape))

X shape (2400, 1, 1, 8) 
 y shape (2400,)


In [ ]:
for epoch in range(5):

  total_loss = 0
  total_correct = 0

  for batch in train_loader: #Get Batch
    images, labels = batch

    preds = network (images) #Pass batch
    loss = F.cross_entropy(preds, labels) #calculate loss

    optimizer.zero_grad()
    loss.backward()  #calculate gradients
    optimizer.step() #update weights

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)

  print("epoch: {0}, Accuracy {1}, loss: {2}".format(epoch, total_correct/len(train_set), loss))

In [17]:
# new training loo

for epoch in range(10):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    
    running_loss = 0.0
    for batch in train_loader:
        
        inputs = batch[0].to("cuda")
        labels = batch[1].to("cuda")

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print (params)
    print ("Training Loss ", running_loss)
    print ("Train - ", evaluate(net, t10_AT_X_train, t10_AT_y_train, params))
#     print ("Validation - ", evaluate(net, X_val, y_val, params))
#     print ("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [16, 1, 1, 8], but got 2-dimensional input of size [50, 8] instead

**Run**

In [71]:
batch_size = 32

for epoch in range(10):  # loop over the dataset multiple times
    print ("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(len(t10_AT_X_train)//batch_size-1): # x_train
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(t10_AT_X_train[s:e]) # x_train
        labels = torch.FloatTensor(np.array([t10_AT_y_train[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print (params)
    print ("Training Loss ", running_loss)
    print ("Train - ", evaluate(net, t10_AT_X_train, t10_AT_y_train, params))
#     print ("Validation - ", evaluate(net, X_val, y_val, params))
#     print ("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0


RuntimeError: Given input size: (4x4x1). Calculated output size: (4x2x0). Output size is too small